In [1]:
from __future__ import division
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

<h2>Gerar população</h2>

In [2]:
# MLP
## Numero de camadas ocultas 1 - 3 (representacao binária 2)
## Numero neuronios 1 - 1000 (representacao binária tamanho 10)

In [3]:
def gerar_populacao(n_ind):
    
    populacao = []
    for i in range(n_ind):
        n_camadas = np.random.randint(1, 4)
        n_neuronios = np.random.randint(1, 1001)
        
        bin_n_camadas = np.binary_repr(n_camadas, width=2)
        bin_n_neuronios = np.binary_repr(n_neuronios, width=10)
        
        crom = list(bin_n_camadas + bin_n_neuronios)
        populacao.append(crom)
    
    return populacao

In [5]:
def binary_converter(arr):
    total = 0
    for index, val in enumerate(reversed(arr)):
        total += (val * 2**index)
    print(total)

<h2>Avaliar Indivíduo</h2>

In [6]:
def avaliar_individuo(individuo, x_train, y_train, x_test, y_test):
    """
        individuo: cromossomo
        x_train: dados para treino
        y_train: labels do treino
        x_test: dados para teste
        y_test: labels do teste
    """
    
    info_n_camadas = int(''.join(individuo[:2]), 2)
    info_n_neuronios = int(''.join(individuo[2:]), 2)
    
    if info_n_camadas == 0 or info_n_neuronios == 0:
        return 0
    else:
        hidden_layers = [info_n_neuronios for i in range(info_n_camadas)]

        mlp = MLPClassifier(hidden_layer_sizes=hidden_layers)
        mlp.fit(x_train, y_train)

        y_pred = mlp.predict(x_test)

        return accuracy_score(y_test, y_pred)

<h2>Crossover</h2>

In [7]:
def crossover(ind1, ind2):
    
    ponto_corte = np.random.randint(1, 11)
    
    ind3 = ind1[:ponto_corte] + ind2[ponto_corte:]
    ind4 = ind2[:ponto_corte] + ind1[ponto_corte:]
    
    return ind3, ind4

<h2>Mutação</h2>

In [8]:
def mutacao(ind):
    
    thresh = 0.1
    
    for i in range(len(ind)):
        v = np.random.rand()
        if v <= thresh:
            ind[i] = str(1 - int(ind[i]))
    
    return ind

<h2>Atualização</h2>

In [9]:
def atualizacao(pop, tam_pop=20, **kwargs):
    
    scores = []
    for ind in pop:
        scores.append(avaliar_individuo(ind, **kwargs))
        
    sort_indexes = np.argsort(scores)
    
    new_pop = []
    
    for i in reversed(range(1, tam_pop+1)):
        idx = sort_indexes[-i]
        new_pop.append(pop[idx])
        
    best_ind = pop[sort_indexes[-1]]
    best_score = scores[sort_indexes[-1]]
    
    return new_pop, best_ind, best_score

<h2>Juntando as partes</h2>

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
# from tqdm import tqdm
from copy import copy

In [11]:
breast_cancer = load_breast_cancer()
x_train, x_test, y_train, y_test = train_test_split(breast_cancer['data'],
                                                       breast_cancer['target'],
                                                       test_size=0.33)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
n_individuos = 20
n_geracoes = 100

populacao = gerar_populacao(n_individuos)

best_ind = None
best_score = None

for gen in range(n_geracoes):
    
    np.random.shuffle(populacao)
    # Selecao
    pais_selecionados = np.random.permutation(n_individuos).reshape(int(n_individuos/2), 2)
    
    # Cruzamento
    for i, j in pais_selecionados:
        populacao = populacao + list(crossover(populacao[i], populacao[j]))
    
    # Mutacao
    for ind in populacao:
        ind = mutacao(ind)
    
    # Atualizacao
    populacao, best_ind_gen, best_score_gen = atualizacao(populacao, 
                                                          tam_pop=n_individuos,
                                                          x_train=x_train, 
                                                          y_train=y_train, 
                                                          x_test=x_test, 
                                                          y_test=y_test)

    
    if best_ind is None or best_score_gen > best_score:
        best_ind = copy(best_ind_gen)
        best_score = best_score_gen
    
    
    info_n_camadas = int(''.join(best_ind[:2]), 2)
    info_n_neuronios = int(''.join(best_ind[2:]), 2)
    print('Melhor modelo até o momento tem {} camadas com {} neurônios em cada'.format(info_n_camadas, info_n_neuronios))
    print('Melhor valor de acurácia até o momento: {}%'.format(np.round(best_score, 2) * 100))
        

c:\users\paula\appdata\local\programs\python\python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Melhor modelo até o momento tem 1 camadas com 252 neurônios em cada
Melhor valor de acurácia até o momento: 99.0%
Melhor modelo até o momento tem 1 camadas com 252 neurônios em cada
Melhor valor de acurácia até o momento: 99.0%
Melhor modelo até o momento tem 1 camadas com 252 neurônios em cada
Melhor valor de acurácia até o momento: 99.0%


c:\users\paula\appdata\local\programs\python\python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
